# 사전 구축된 에이전트 사용

자세한 내용은 [Start with a prebuilt agent](https://langchain-ai.github.io/langgraph/agents/agents/) 페이지를 참조

## 1. 환경설정

In [1]:
# 환경 변수 확인
from dotenv import load_dotenv
import os
load_dotenv(override=True)

# TAVILY_API_KEY= os.environ.get("TAVILY_API_KEY")
# print(TAVILY_API_KEY[:20])

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")
_set_env("OPENAI_API_KEY")

## 2. 기본 에이전트 생성

In [4]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """주어진 도시의 날씨를 확인합니다."""
    return f"{city}의 날씨는 항상 화창합니다!"

agent = create_react_agent(
    model="openai:gpt-4o-mini",  
    temperature=0,
    tools=[get_weather],  
    prompt="당신은 유용한 AI 도우미입니다."  
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울의 날씨는 어떤가?"}]}
)

response["messages"][-1].content

'서울의 날씨는 항상 화창합니다! 추가로 확인하고 싶은 정보가 있나요?'

## 2. 동적 프롬프트

In [12]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"당신은 유용한 AI 도우미입니다. 사용자에게 {user_name}이라고 호칭하세요."
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather],
    prompt=prompt
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울의 날씨는 어떤가?"}]},
    config={"configurable": {"user_name": "제임스"}}
)

response["messages"][-1].content

'서울의 날씨는 항상 화창합니다! 다른 정보가 필요하시면 말씀해 주세요, 제임스.'

## 3. 메모리 추가

In [13]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model

checkpointer = InMemorySaver()

def get_weather(city: str) -> str:  
    """주어진 도시의 날씨를 확인합니다."""
    return f"{city}의 날씨는 항상 화창합니다!"

model = init_chat_model(
    "openai:gpt-4o-mini",
    temperature=0
)

agent = create_react_agent(
    model=model,  
    tools=[get_weather],  
    checkpointer=checkpointer,
    prompt="당신은 유용한 AI 도우미입니다."  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
response = agent.invoke(
    {"messages": [{"role": "user", "content": "안녕 내이름은 제이슨이야?"}]},
    config  
)

response["messages"][-1].content

'안녕하세요, 제이슨! 어떻게 도와드릴까요?'

In [14]:
print(response["messages"][-1].content)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "내이름이 뭐라고 했지?"}]},
    config
)

response["messages"][-1].content

안녕하세요, 제이슨! 어떻게 도와드릴까요?


'당신의 이름은 제이슨입니다.'

## 4. 구조화 된 출력

In [16]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울의 날씨는 어떤가?"}]}
)

response["structured_response"]

WeatherResponse(conditions='서울의 날씨는 항상 화창합니다!')